Agregar la librería de selenium

In [139]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
chop = Options()
chop.add_argument("headless=new")

driver = webdriver.Chrome(options=chop)



In [140]:
ENTIDADES_URL = "https://es.wikipedia.org/wiki/Anexo:Entidades_federativas_de_M%C3%A9xico_por_superficie,_poblaci%C3%B3n_y_densidad"
driver.get(ENTIDADES_URL)
#Get first 3 tables to avoid the footer
tables = driver.find_elements(By.CSS_SELECTOR,"table")[0:3]


In [141]:
rows = tables[0].find_elements(By.CSS_SELECTOR , "tr")[3:]
# From row 3 and below the elements are entries
print(rows[0].get_attribute("innerHTML"))


<td bgcolor="#d8d8d8">01</td>
<td bgcolor="#d8d8d8">11</td>
<td bgcolor="#d8d8d8">30</td>
<td><a href="/wiki/Chihuahua" title="Chihuahua">Chihuahua</a></td>
<td align="right">247&nbsp;455</td>
<td align="right">12,6</td>
<td align="right">3 741 869</td>
<td align="right">14</td>
<td bgcolor="#f8f8f0"><a href="/wiki/Chihuahua_(Chihuahua)" title="Chihuahua (Chihuahua)">Chihuahua</a></td>
<td align="right" bgcolor="#f8f8f0">925 762</td>
<td bgcolor="#f8f8f0">05
</td>


Entidades federativas de México por superficie, población y densidad

In [47]:
import re

def get_numbers(string_item):
    return re.sub("[^\d\.]", "" ,string_item)

for row in rows:
    # Different DOM structure
    estado = row.find_element(By.CSS_SELECTOR,"td>a").get_attribute("innerHTML")
    #Rest of the items
    cells = row.find_elements(By.CSS_SELECTOR, "td")
    superficie = get_numbers(cells[4].get_attribute("innerHTML"))
    poblacion = get_numbers(cells[6].get_attribute("innerHTML"))
    densidad = get_numbers(cells[7].get_attribute("innerHTML"))

    
    print(f"{estado} : {superficie} | {poblacion} | {densidad}")


Chihuahua : 247455 | 3741869 | 14
Sonora : 179355 | 2944840 | 16
Coahuila : 151562 | 3146771 | 19
Durango : 123317 | 1832650 | 14
Oaxaca : 93757 | 4132148 | 42
Tamaulipas : 80249 | 3527735 | 43
Jalisco : 78588 | 8348151 | 100
Zacatecas : 75284 | 1622138 | 21
Baja California Sur : 73909 | 798447 | 10
Chiapas : 73311 | 5543828 | 71
Veracruz  : 71826 | 8062579 | 113
Baja California : 71450 | 3769020 | 46
Nuevo León : 64156 | 5784442 | 80
Guerrero : 63596 | 3540685 | 56
San Luis Potosí : 61137 | 2822255 | 44
Michoacán : 58599 | 4748846 | 78
Sinaloa : 58200 | 3026943 | 52
Campeche : 57507 | 928363 | 16
Quintana Roo : 44705 | 1857985 | 34
Yucatán : 39524 | 2320898 | 53
Puebla : 34306 | 6583278 | 180
Guanajuato : 30608 | 6166934 | 191
Nayarit : 27857 | 1235456 | 42
Tabasco : 24731 | 2402598 | 97
Estado de México : 22351 | 16992418 | 724
Hidalgo : 20813 | 3082841 | 137
Querétaro : 11699 | 2368467 | 174
Colima : 5627 | 731391 | 126
Aguascalientes : 5616 | 1425607 | 234
Morelos : 4879 | 1971520 

Censos (INEGI) 1900-2020

In [61]:
rows = tables[1].find_elements(By.CSS_SELECTOR , "tr")[2:]

# YEAR 2020 is cell 2 and above
from collections import defaultdict
def default_value():
    return []

dtable2 = defaultdict(default_value)
# Assume the population list is from 2020 - 10 * iteration
for row in rows:
    # Un estado por fila
    estado = row.find_element(By.CSS_SELECTOR,"td>a").get_attribute("innerHTML")
    for cell in row.find_elements(By.CSS_SELECTOR , "td")[2:]:
        poblacion_value = get_numbers(cell.get_attribute("innerHTML"))
        dtable2[estado].append(poblacion_value)

for key , value in dtable2.items():
    print(f"{key} : {value}")

Estado de México : ['16992418', '15175862', '13096686', '9815795', '7564335', '3833185', '1897851', '1392623', '1146340', '990112', '884617', '989510']
Ciudad de México : ['9209944', '8851080', '8605239', '8235744', '8831079', '6874165', '4870876', '3050442', '1757530', '1229576', '906063', '720753']
Jalisco : ['8348151', '6322002', '5302689', '4371998', '3296586', '2443261', '1746777', '1418310', '1255346', '1191957', '1208855', '1137311']
Veracruz : ['8062579', '6908975', '6228239', '5387680', '3815422', '2727899', '2040231', '1619338', '1377293', '1159935', '1132859', '960570']
Puebla : ['6583278', '5076686', '4126101', '3347685', '2508226', '1973837', '1625830', '1294620', '1150425', '1024955', '1101600', '1024446']
Guanajuato : ['6166934', '4663032', '3982593', '3006110', '2270370', '1735490', '1328712', '1046490', '987801', '860364', '1081651', '1065317']
Nuevo León : ['5784442', '3834141', '3098736', '2513044', '1694689', '1078848', '740191', '541147', '417491', '336412', '36515

Proyecciones de población 2010-2030 (CONAPO)

In [76]:
rows = tables[2].find_elements(By.CSS_SELECTOR , "tr")[2:-1]

href_to_visit = []

# YEAR 2020 is cell 2 and above
dtable3 = defaultdict(default_value)
# Assume the population list is from 2020 - 10 * iteration
for row in rows:
    estado = row.find_element(By.CSS_SELECTOR, "td>a")
    href_to_visit.append( estado.get_attribute("href") )
    
    for cell in row.find_elements(By.CSS_SELECTOR, "td"):
        poblacion_value = get_numbers(cell.get_attribute("innerHTML"))
        dtable3[estado.get_attribute("innerHTML")].append(poblacion_value)


for key , value in dtable2.items():
    print(f"{key} : {value}")

Estado de México : ['15571679', '16870388', '18075065', '19178922', '20167433', '15571679', '16870388', '18075065', '19178922', '20167433']
Ciudad de México : ['8944599', '8854600', '8738914', '8600179', '8439786', '8944599', '8854600', '8738914', '8600179', '8439786']
Veracruz : ['7712247', '8046828', '8328389', '8573860', '8781620', '7712247', '8046828', '8328389', '8573860', '8781620']
Jalisco : ['7442625', '7931267', '8363277', '8751124', '9102259', '7442625', '7931267', '8363277', '8751124', '9102259']
Puebla : ['5863823', '6193836', '6481536', '6730952', '6942481', '5863823', '6193836', '6481536', '6730952', '6942481']
Guanajuato : ['5558502', '5817614', '6033559', '6214047', '6361401', '5558502', '5817614', '6033559', '6214047', '6361401']
Chiapas : ['4933755', '5252808', '5568648', '5861450', '6129218', '4933755', '5252808', '5568648', '5861450', '6129218']
Nuevo León : ['4923273', '5085848', '5440277', '5778562', '6097769', '4923273', '5085848', '5440277', '5778562', '6097769'

Utilizar las tablas de HREF to visit

In [136]:

dtoponimias = defaultdict()

for link in href_to_visit:
    driver.get(link)

    text_area = driver.find_element(By.CSS_SELECTOR, "#mw-content-text>div")
    paragraphs = text_area.find_elements(By.XPATH, "./*")

    is_toponimia_next = False

    for paragraph in paragraphs:
        if paragraph.tag_name == "h2":
            try:
                titulo = paragraph.find_element(By.ID, "Toponimia").get_attribute("innerHTML")
                is_toponimia_next = True
                continue
            except:
                pass
        if is_toponimia_next:
            # Avoid figures or spans embedded between title and text
            if paragraph.tag_name != "p":
                continue
            print(link)
            print(paragraph.text)  
            dtoponimias[link] = paragraph.text
            break



https://es.wikipedia.org/wiki/Estado_de_M%C3%A9xico
El nombre del Estado de México deriva del término náhuatl Mēxihco [me:ʃiʔko], que designaba la capital de los mexicas, México-Tenochtitlan. De acuerdo con Bernardino de Sahagún, el vocablo significa ‘En el ombligo de la luna’.20 La hipótesis ha sido objetada porque la morfología del náhuatl no admite una derivación del topónimo a partir de las voces propuestas.21 Francisco Javier Clavijero sugería que el topónimo debía interpretarse como ‘Lugar de Mexihtli’, es decir, de Huitzilopochtli, pues Mexihtli era uno de sus nombres alternativos.22
https://es.wikipedia.org/wiki/Veracruz_de_Ignacio_de_la_Llave
El nombre del estado se debe a la cuatro veces heroica ciudad y puerto de Veracruz, fundada por Hernán Cortés el 22 de abril de 1519, como la Villa Rica de la Vera Cruz dicho nombre tiene el siguiente origen: Villa por ser parecido a las villas españolas; Rica por la cantidad de oro obtenido de los indígenas y Vera Cruz debido a la fecha 

In [138]:
for key ,value in dtoponimias.items():
    print(f"{key} : {value}")

len(dtoponimias)

https://es.wikipedia.org/wiki/Estado_de_M%C3%A9xico : El nombre del Estado de México deriva del término náhuatl Mēxihco [me:ʃiʔko], que designaba la capital de los mexicas, México-Tenochtitlan. De acuerdo con Bernardino de Sahagún, el vocablo significa ‘En el ombligo de la luna’.20 La hipótesis ha sido objetada porque la morfología del náhuatl no admite una derivación del topónimo a partir de las voces propuestas.21 Francisco Javier Clavijero sugería que el topónimo debía interpretarse como ‘Lugar de Mexihtli’, es decir, de Huitzilopochtli, pues Mexihtli era uno de sus nombres alternativos.22
https://es.wikipedia.org/wiki/Veracruz_de_Ignacio_de_la_Llave : El nombre del estado se debe a la cuatro veces heroica ciudad y puerto de Veracruz, fundada por Hernán Cortés el 22 de abril de 1519, como la Villa Rica de la Vera Cruz dicho nombre tiene el siguiente origen: Villa por ser parecido a las villas españolas; Rica por la cantidad de oro obtenido de los indígenas y Vera Cruz debido a la fe

30